This sort intro uses Keras for :
1. Uses Keras to load predefined dataset
2. Create an automatic neureal network machine learning model that classifies images
3. Train this neural network
4. Evaluate the accuracy of the model

Configure TensorFlow
Start by importing TensorFlow


In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import numpy as np
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.0


## Load the dataset

Load the MNIST dataset. Convert the whole number examples in flating point number.
The pixel values of the images range from 0 trough 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also convert from int to float

In [2]:
import sys
import numpy

numpy.set_printoptions(threshold=sys.maxsize)

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # These are matrices the max value is 255 so we divide by 255 to get values float values between 0 and 1

## Build a machine learning model

Create an automatic learning model
Create a model tf.keras.Sequential by stacking layers 

1. I Keras, there is 2 ways of building models : 

    - **Sequential** model : Allow to build model layer by layer. Does not allow to create models that share layers or have multiple inputs or outputs.
    - **Functional** model : Allow to create fare more complexe models. Layers are not only connected to the previous and the next one but can be connected to any other one. 
    
        This allow to create networks such as siamese or residual networks
    - So Sequential is useful for stacking layers where each layer has one input **tensor** and one output **tensor**
    - A **tensor** is an array in Tensorflow. It is like a np.arrays in NumPy. Basically, they are used like matrices in the neural networks computations. they can be rank-0 (constant array), rank-1 (1-D array), rank-2 (2-D array), etc...
    - Layers are function with a known mathematical structure that can be reused and have trainable variables. Most Tensorflow models are composed of layers.
    - **Flatten layer** : collapses the spatial dimension of the input into the channel dimension. Ex : input = H by W by C by N by S array (sequence oif images), then the flattened output is (H\*W\*B) by N by S array. 

        In our example, the Dense layer is using a 128-d vector so the Flatten layer will automaticcaly transform the input to match the requirements of the Dense layer
    - **Dense layer** : The Dense layer is a neural network that is **connected deeply**. This means that each neuron in the dense layer is receives input from all neurons of its previous layer. Most commonly used layer in the models. 
        
        In the background, the dense layer performs a matric-vector multiplication. The values used in the matrix are actually parameters that can be trained and updated with the help of backpropagation. 
        
            keras.layers.Dense(units, activation=None, use_bias=True, ...) 

                - units = represents the output size of the layer. It is the unit parameter itself that plays a major role in the size of the weight matrix along the basic vector 

                - activation = activation function = function that decides wether a neuron should be activated or not by calculating the weighted sum and further adding bias to it. 

                **Weight*** increase the steepness of activation function = (how fast the activation function will trigger). Whereas the **Bias** is used to delay the triggerinf of the activation function. 
                It may be critical for successful training.
                
                The purpose is to introduce a non-linearity into the output of a neuron. Makes the back-propagation possible. 
                There is a lot of different activation functions with differnets outcomes and roles.  

                - use_bias = wether we should use bias or not, default is True. 

    - **Dropout layer** : randomly sets input units to 0 with a frequency of rate at each step during training. = Drop some inputs, which help prevent overfitting. 

        - overfitting = a model that models the training data too well = rely too much on the training data instead of learning how to find output by itself. 
        - underfitting = a model that can neither model the training data nor generalize to new data. 
        - **Ideally, we want a model at the sweet spot between overfitting and underfitting.**
    

In [3]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________



- The model returns a vector of **logits** or **log-odds** : 
    - **logits** : The vector of raw (= non-normalized) predictions that the classification model generates. **KEEP IN MIND THAT THE PREDICTIONS HAVE TO BE NORMALIZED USING A NORMALIZATION FUNCTION (LIKE SOFTMAX ?) AFTERWARD**
    - **log-odds**: The logarithm of the odds of some events.

In [4]:
predictions = model(x_train[:1]).numpy() # !! we need to pass an array to the model so we can NOT call it with x_train[0] since it will returns only the value, this is why we use x_train[:1]. [3, 4, 5][0] = 3 WHEREAS [3, 4, 5][:1] = [3]
predictions

array([[ 0.19953391, -0.13593133,  0.56630874,  0.49914253,  0.08583802,
         0.04749148,  0.37313107,  0.0812885 ,  0.30607626, -0.6289251 ]],
      dtype=float32)

The _tf.nn.softmax_ functon converts these logits to probabilities for each class. This is the normalization function.

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.10115221, 0.07232438, 0.14597003, 0.1364878 , 0.09028132,
        0.08688489, 0.12032827, 0.08987152, 0.11252425, 0.04417537]],
      dtype=float32)

**Note: It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.**

Define a loss unfciton for training using **losses.SparseCategoricalCrossentropy**


In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values (= real value from dataset, value we know to be true) and a vector of logits anrd returns a scalar loss for each example. 
This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.
This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to ```tf.math.log(1/10) ~= 2.3```

In [7]:
loss_fn(y_train[:1], predictions).numpy()

[[ 0.19953391 -0.13593133  0.56630874  0.49914253  0.08583802  0.04749148
   0.37313107  0.0812885   0.30607626 -0.6289251 ]]


2.4431713

Before you start training, configure and compile the model using Keras ``Model.compile``. Set the optimize class to ``"adam"``, set the the ``loss`` ``loss_fn`` function you defined earlier, and specify a metric to be evaluated for the model by settings the ``metrics`` parameters to ``accuracy``.

Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure to update network weights iterative based in training data.


In [8]:
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

## Train and evaluate your model

Use the ``Model.fit`` method to adjust your model parameters and minimize the loss:

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3057 - accuracy: 0.9106
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1473 - accuracy: 0.9558
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1104 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0896 - accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0748 - accuracy: 0.9769


The ``Model.evaluate`` method checks the model's performance, usually on a validation set or test set.

In [10]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0861 - accuracy: 0.9746 - 903ms/epoch - 3ms/step


[0.08612670749425888, 0.9746000170707703]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it :

In [11]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.9309323e-06, 1.2683474e-10, 5.4431612e-06, 5.1775336e-05,
        5.2585852e-11, 8.6468347e-08, 3.0214440e-12, 9.9993265e-01,
        5.5024614e-08, 8.0744230e-06],
       [3.4143693e-07, 1.3269081e-04, 9.9964666e-01, 2.1180366e-04,
        2.2969859e-15, 5.3869958e-06, 1.1500309e-06, 1.3457478e-13,
        2.0551868e-06, 6.7652771e-12],
       [2.0512924e-07, 9.9895740e-01, 7.4604654e-06, 1.9420379e-05,
        4.7965521e-05, 6.2324416e-06, 3.0984390e-06, 8.0779148e-04,
        1.4771392e-04, 2.6819459e-06],
       [9.9995613e-01, 1.2489338e-11, 1.2760277e-05, 8.7295876e-10,
        2.0110861e-08, 2.9480293e-07, 3.0191983e-05, 4.8356782e-07,
        2.0667862e-10, 1.3822645e-07],
       [1.2937604e-06, 6.8424879e-09, 6.3373140e-05, 2.8516661e-07,
        9.8487502e-01, 1.2192855e-06, 5.9674181e-07, 1.3881865e-04,
        3.3172171e-06, 1.4916180e-02]], dtype=float32)>

In [33]:
for i in range(100):
    q = model.predict( np.array([x_test[i]]) )

    print(f"q len = {len(q)}")
    print(f"result = {y_test[i]}" )
    print(f"found result = {np.where(q[0] == numpy.amax(q[0]))[0][0] }")
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####
    ####§§§TODO REVOIR TUTO FLEUR, QUELLE EST LA VALEURE D'UN NEURONE ? puis faire autre tuto nmis tensorflow pour mieux comprendre
    ####

1/1 [==============================] - 0s 20ms/step
q len = 1
result = 7
found result = 7
1/1 [==============================] - 0s 17ms/step
q len = 1
result = 2
found result = 2
1/1 [==============================] - 0s 17ms/step
q len = 1
result = 1
found result = 1
1/1 [==============================] - 0s 19ms/step
q len = 1
result = 0
found result = 0
1/1 [==============================] - 0s 17ms/step
q len = 1
result = 4
found result = 4
1/1 [==============================] - 0s 19ms/step
q len = 1
result = 1
found result = 1
1/1 [==============================] - 0s 19ms/step
q len = 1
result = 4
found result = 4
1/1 [==============================] - 0s 19ms/step
q len = 1
result = 9
found result = 9
1/1 [==============================] - 0s 17ms/step
q len = 1
result = 5
found result = 5
1/1 [==============================] - 0s 17ms/step
q len = 1
result = 9
found result = 9
1/1 [==============================] - 0s 18ms/step
q len = 1
result = 0
found result = 0
1/1 [=====